# Working with images

There is an astropy package devoted entirely to FITS, but if you are working with images it is easier to use CCDData to create the images.

## CCDData as a container for CCD images

A `CCDData` object can store the data, mask and metadata for an image.

In [ ]:
import numpy as np

from astropy.utils.data import get_pkg_data_filename
from astropy.nddata import CCDData

%matplotlib inline
import matplotlib.pyplot as plt


### Download image to work with (65MB)

In [ ]:
ccd_for_saving = CCDData.read('http://physics.mnstate.edu/craig/ey-uma-S001-R001-C037-rp.fit.gz')
ccd_for_saving.write('ey-uma-S001-R001-C037-rp.fit', overwrite=True)

In [ ]:
ccd = CCDData.read('ey-uma-S001-R001-C037-rp.fit')

The header or metadata is available via `ccd.header` or `ccd.meta`.

In [ ]:
ccd.header

For this file, the mask and uncertainty are None.

In [ ]:
ccd.mask

In [ ]:
ccd.uncertainty

In [ ]:
ccd.wcs

Every `CCDData` object has to have a unit.

In [ ]:
ccd.unit

In [ ]:
ccd.uncertainty

Populate the mask and uncertainty

In [ ]:
ccd.mask = ccd.data > 38000

Assuming a read noise of 2.9 electrons, and a gain of 15 electrons per ADU, compute an uncertainty.

In [ ]:
read_noise = 2.9
gain = 15
ccd.uncertainty = np.sqrt(read_noise**2 + gain*ccd.data) / gain

Subtract off a DC offset

In [ ]:
ccd.data = ccd.data - np.median(ccd.data)

Write out to a file

In [ ]:
ccd.write('ccddata.fits', overwrite=True)

Read back in

In [ ]:
ccd_new = CCDData.read('ccddata.fits')

In [ ]:
plt.figure()
plt.imshow(ccd_new.data)
plt.colorbar()

### Astronomical images often need some scaling

One convenient way of doing that is the `AsymmetricPercentileInterval`, which lets you specify the lower and upper percentile of pixels that will be mapped to each end of the color spectrum.

In [ ]:
from astropy.visualization import AsymmetricPercentileInterval, ImageNormalize

In [ ]:
interval = AsymmetricPercentileInterval(25, 99) 

In [ ]:
maplotlib_norm = ImageNormalize(ccd_new.data, interval=interval)
plt.figure()
plt.imshow(ccd_new.data, origin='lower', norm=maplotlib_norm)
plt.colorbar()

## Interactive images with `astrowidgets`

The `astrowidgets` package provides an interactive image view for a Jupyter notebook. At the moment the import statement is a little complicated and depends on which version of astrowidgets you have.

[astrowidgets](https://astrowidgets.readthedocs.io/en/latest/install.html) builds on ipywidgets library to provide an in-browser interactivate image viewer. Expect some of its details to evolve over the next year.

[ginga](https://ginga.readthedocs.io/en/latest/) is an image viewing library that can use a bunch of ddifferent backends

In [ ]:
# The correct import depends on what version of astrowidgets is installed,
# unfortunately.

try:
    from astrowidgets.ginga import ImageWidget
except ImportError:
    from astrowidgets import ImageWidget

### astrowidgets controls

Use the `+` and `-` keys to zoom in and out. Either click and drag to pan or scroll to pan.

In [ ]:
iw = ImageWidget()
iw

Load an image, option 1

In [ ]:
iw.load_nddata(ccd_new)

If we hadn't already read in the file you can do this instead

In [ ]:
iw.load_fits('ey-uma-S001-R001-C037-rp.fit')